<a href="https://colab.research.google.com/github/arunmergu/DeepLearning/blob/main/Unsupervised%20techniques%20using%20DL/Text_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

In [2]:
!wget https://www.dropbox.com/s/kfye28it99g9j12/imdb_labelled.txt

--2021-03-03 23:02:12--  https://www.dropbox.com/s/kfye28it99g9j12/imdb_labelled.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/kfye28it99g9j12/imdb_labelled.txt [following]
--2021-03-03 23:02:12--  https://www.dropbox.com/s/raw/kfye28it99g9j12/imdb_labelled.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc51113844ed81f39eb061347f47.dl.dropboxusercontent.com/cd/0/inline/BKBBoIVsw_uZCtC-5xpyCa4H1UC7LQWMW8a4AQT9p2TJ6Kt9Vj7kiaRpo0pk4RMavXWH2sxAMuvDGZtPEAYkfaABEcN_dpKqExq7fMv-m9mkPg/file# [following]
--2021-03-03 23:02:13--  https://uc51113844ed81f39eb061347f47.dl.dropboxusercontent.com/cd/0/inline/BKBBoIVsw_uZCtC-5xpyCa4H1UC7LQWMW8a4AQT9p2TJ6Kt9Vj7kiaRpo0pk4RMavXWH2sxAMuvDGZtPEAYkfaABEcN_dpKqExq7fMv-m9mkPg/file
R

In [3]:
file=r'/content/imdb_labelled.txt'

In [4]:
import pandas as pd
imdb = pd.read_csv(file,sep='\t',header=None,names=['review','target'])

In [5]:
# you can download glove embeddings from here directly using wget
# just copy the link for the embeddings that you want to use 
# https://nlp.stanford.edu/projects/glove/

In [6]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip
# downloading and unzipping will take some time 

--2021-03-03 23:02:13--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2021-03-03 23:02:13--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2021-03-03 23:02:14--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [7]:
!unzip /content/glove.42B.300d.zip

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [8]:
!rm /content/glove.42B.300d.zip

In [9]:
import numpy as np

In [10]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()
# this again will take good amount of time

In [11]:
from sklearn.model_selection import train_test_split
data_train,data_test=train_test_split(imdb,test_size=0.2)

In [12]:
x_train=data_train['review']
y_train=data_train['target']
x_test=data_test['review']
y_test=data_test['target']

In [29]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM,Activation,Dense,Input,Embedding,Dropout
from tensorflow.keras.models import Model,Sequential

In [14]:
from nltk import word_tokenize

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
sent_lens=[len(word_tokenize(x)) for x in x_train]

In [17]:
np.percentile(sent_lens,95)

40.0

In [18]:
max_len=40

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [19]:
# creating our own embedding matrix to bring down the size to 300
# we'll use 300 D vector representation of the words from pretrained embedding index 
# that we downloaded 

embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector
        
# if there are specific words which are not present in pretrained embedding 
# their weights will remain 0. if there are too many such words 
# then you should probably not use pretrained embeddings 

In [48]:
# Now while creating the model for text data , we'll keep embedding layer weights 
# non-trainable as we are supplying those values from our end 
model1=Sequential()
model1.add(Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False))
model1.add(LSTM(50))
model1.add(Dense(10,activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')

In [44]:
# you can see that bulk of the weights come from embedding layer
# using pretrained weights for the same , brings down number of trainable parameters 
# by a huge amount. This in turn results in pretty short training times 
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 40, 300)           844800    
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                70200     
_________________________________________________________________
dense_6 (Dense)              (None, 10)                510       
_________________________________________________________________
dropout_3 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 11        
Total params: 915,521
Trainable params: 70,721
Non-trainable params: 844,800
_________________________________________________________________


In [49]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [50]:
model1.fit(seq_train_matrix,y_train.values,validation_split=0.2,epochs=40,class_weight={0:1,1:12},batch_size=50)

Epoch 1/40
10/10 [==============================] - 5s 221ms/step - loss: 4.5755 - accuracy: 0.5316 - val_loss: 0.7199 - val_accuracy: 0.5250
Epoch 2/40
10/10 [==============================] - 1s 93ms/step - loss: 2.7753 - accuracy: 0.5017 - val_loss: 0.9472 - val_accuracy: 0.5250
Epoch 3/40
10/10 [==============================] - 1s 90ms/step - loss: 2.0782 - accuracy: 0.5210 - val_loss: 1.2028 - val_accuracy: 0.5250
Epoch 4/40
10/10 [==============================] - 1s 84ms/step - loss: 1.9367 - accuracy: 0.4872 - val_loss: 1.2681 - val_accuracy: 0.5250
Epoch 5/40
10/10 [==============================] - 1s 90ms/step - loss: 1.9059 - accuracy: 0.4905 - val_loss: 1.2087 - val_accuracy: 0.5250
Epoch 6/40
10/10 [==============================] - 1s 87ms/step - loss: 1.8894 - accuracy: 0.5299 - val_loss: 1.1483 - val_accuracy: 0.5250
Epoch 7/40
10/10 [==============================] - 1s 88ms/step - loss: 1.8802 - accuracy: 0.5102 - val_loss: 1.1334 - val_accuracy: 0.5250
Epoch 8/40
1

In [51]:
p=model1.predict(seq_test_matrix)

In [52]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,p)

0.8901353276353277